# Aggregation for 15 km (9 grids of 5 km)

In [1]:
import netCDF4
import numpy as np
import pandas as pd
import requests
import os
import pathlib
import shutil
import sys
import time
import calendar
import datetime
from numpy  import array
from glob import glob

# Merging the climate variables into a single file (define coordinates)

In [2]:
#Specify latitude of the left top corner in the grid to aggregate
a=-43.25
#Specify longitude of the left top corner in the grid to aggregate
b=146.90
#Some calculation...
a2=round(a-0.05,2)
a3=round(a-0.1,2)
b2=round(b+0.05,2)
b3=round(b+0.1,2)

#Defining the ranges
lat_range=[a3,a2,a]
lon_range=[b,b2,b3]

dirname= r'C:\Users\jjojeda.000\Dropbox\DATA\APSIM sim\base climate data'

#Merge individuals variables into a single file
def generate_dataframe(lat_range, lon_range, download_files=False, output_to_file=True, create_met=False):

    for lat in lat_range:

            for lon in lon_range:
                
                df1= pd.read_csv(os.path.join(dirname,'{}-{}-{}.met'.format("radiation",lat,lon)))
                df2= pd.read_csv(os.path.join(dirname,'{}-{}-{}.met'.format("max_temp",lat,lon)))
                df3= pd.read_csv(os.path.join(dirname,'{}-{}-{}.met'.format("min_temp",lat,lon)))
                df4= pd.read_csv(os.path.join(dirname,'{}-{}-{}.met'.format("daily_rain",lat,lon)))
                df5=df1.merge(df2)
                df6=df5.merge(df3)
                dffinal=df6.merge(df4)

                #Change name of variables
                dffinal.rename(columns={'radiation': 'radn','max_temp': 'maxt','min_temp': 'mint','daily_rain': 'rain'}, inplace=True)

                #Delete the head of the following years
                dffinal2=dffinal[dffinal.year != 'year']

                #Export combined csv
                dffinal2.to_csv('{}-{}.met'.format(lat,lon), sep=" ", index=False, mode="a")
                
generate_dataframe(lat_range=lat_range, lon_range=lon_range)

# Creating the aggregated climate file

In [3]:
#Some explanation:
#for a given grid of 15*15km, we have the corresponding coordinates:
#lat_range=[-39.8,-39.75,-39.7] == lat_range=[0,1,2]
#lon_range=[143.85,143.9,143.95] == lon_range=[0,1,2]

#Open grid files
f1 = pd.read_csv('{}-{}.met'.format(lat_range[0],lon_range[0]), sep=" ", index_col=False)
f2 = pd.read_csv('{}-{}.met'.format(lat_range[0],lon_range[1]), sep=" ", index_col=False)  
f3 = pd.read_csv('{}-{}.met'.format(lat_range[0],lon_range[2]), sep=" ", index_col=False)  
f4 = pd.read_csv('{}-{}.met'.format(lat_range[1],lon_range[0]), sep=" ", index_col=False)  
f5 = pd.read_csv('{}-{}.met'.format(lat_range[1],lon_range[1]), sep=" ", index_col=False)  
f6 = pd.read_csv('{}-{}.met'.format(lat_range[1],lon_range[2]), sep=" ", index_col=False)  
f7 = pd.read_csv('{}-{}.met'.format(lat_range[2],lon_range[0]), sep=" ", index_col=False)  
f8 = pd.read_csv('{}-{}.met'.format(lat_range[2],lon_range[1]), sep=" ", index_col=False)  
f9 = pd.read_csv('{}-{}.met'.format(lat_range[2],lon_range[2]), sep=" ", index_col=False)  

#Merge grids files
merged_table=pd.concat([f1,f2,f3,f4,f5,f6,f7,f8,f9], axis=1, sort=False)

#Calculate average of each variable across the grid files
merged_table['radn_avg'] = merged_table[['radn']].mean(axis=1).round(decimals=1)
merged_table['maxt_avg'] = merged_table[['maxt']].mean(axis=1).round(decimals=1)
merged_table['mint_avg'] = merged_table[['mint']].mean(axis=1).round(decimals=1)
merged_table['rain_avg'] = merged_table[['rain']].mean(axis=1).round(decimals=1)

#Delete undesirable columns 
merged_table.drop(columns=['radn','maxt','mint','rain'], axis=1, inplace=True)

#Delete duplicated columns
merged_table = merged_table.loc[:,~merged_table.columns.duplicated()]

#Change name of variables
merged_table.rename(columns={'radn_avg': 'radn','maxt_avg': 'maxt','mint_avg':'mint','rain_avg':'rain'}, inplace=True)

#Remove rubbish
os.remove('{}-{}.met'.format(lat_range[0],lon_range[0]))
os.remove('{}-{}.met'.format(lat_range[0],lon_range[1]))
os.remove('{}-{}.met'.format(lat_range[0],lon_range[2]))
os.remove('{}-{}.met'.format(lat_range[1],lon_range[0]))
os.remove('{}-{}.met'.format(lat_range[1],lon_range[1]))
os.remove('{}-{}.met'.format(lat_range[1],lon_range[2]))
os.remove('{}-{}.met'.format(lat_range[2],lon_range[0]))
os.remove('{}-{}.met'.format(lat_range[2],lon_range[1]))
os.remove('{}-{}.met'.format(lat_range[2],lon_range[2]))

# Creating the final apsim met file (including head, amp and tav)

In [4]:
#Creating the head for the met apsim file:

#Get the months as a column
merged_table = merged_table.convert_objects(convert_numeric=True)
merged_table['cte']=1997364
merged_table['day2']=merged_table['day']+merged_table['cte']
merged_table['date'] = (pd.to_datetime((merged_table.day2 // 1000)) + pd.to_timedelta(merged_table.day2 % 1000, unit='D'))
merged_table['month'] = merged_table.date.dt.month
month=merged_table.loc[:,'month']

#Calculate amp
merged_table['tmean'] = merged_table[['maxt', 'mint']].mean(axis=1)
tmeanbymonth=merged_table.groupby(month)[["tmean"]].mean()
maxmaxtbymonth=tmeanbymonth.loc[tmeanbymonth['tmean'].idxmax()].round(decimals=5)
minmaxtbymonth=tmeanbymonth.loc[tmeanbymonth['tmean'].idxmin()].round(decimals=5)
a=maxmaxtbymonth-minmaxtbymonth
b=list(a)
am=array(b).round(decimals=5)
for i in am:
    print(i)

#Calculate tav
#tav=merged_table["tmean"].mean()
tav=tmeanbymonth.groupby(month)[["tmean"]].mean().round(decimals=5)
z=tav.iloc[0]['tmean']

#Delete some rubbish
merged_table.drop(merged_table.columns[[6, 7, 8, 9, 10]], axis=1, inplace=True)

#Add variables units:
#create series from types_header_for_insert
merged_final_file = pd.DataFrame(columns = merged_table.columns)
s = pd.Series(["()","()","(MJ^m2)","(oC)","(oC)","(mm)"], index=merged_table.columns)
merged_final_file = merged_final_file.append(s, ignore_index=True).append(merged_table, ignore_index=True)

#Write head of file
with open('{}-{}.met'.format(lat_range[2],lon_range[0]),'a') as file:
        file.write('[weather.met.weather]\n')
        file.write('                         !station number={}-{}\n'.format(lat_range[2],lon_range[0]))
        file.write('                         Latitude={}\n'.format(lat_range[2]))
        file.write('                         Longitude={}\n'.format(lon_range[0]))
        file.write('                         tav={}\n'.format(z))
        file.write('                         amp={}\n\n'.format(i))
        
#Export combined csv
merged_final_file.to_csv('{}-{}.met'.format(lat_range[2],lon_range[0]), sep=" ", index=False, mode="a")

7.98372


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.
